In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# Experiment configuration
experiment_name = "fusion_7_n100"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [3]:
import torch
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm_notebook

import fannypack
from lib import panda_datasets, panda_baseline_models, panda_baseline_training
from lib.ekf import KalmanFilterNetwork
from fannypack import utils
from lib import dpf
from lib.panda_models import PandaDynamicsModel, PandaEKFMeasurementModel

from lib.fusion import KalmanFusionModel
from lib.fusion import CrossModalWeights

import lib.panda_kf_training as training

In [4]:
print("Creating dataset...")
# dataset_full = panda_datasets.PandaParticleFilterDataset(
#     'data/gentle_push_10.hdf5',
#     subsequence_length=16,
#     **dataset_args)

e2e_trainset = panda_datasets.PandaParticleFilterDataset(
    "data/gentle_push_100.hdf5",
    subsequence_length=16,
    particle_count=1,
    particle_stddev=(.03, .03),
    **dataset_args
)

dataset_measurement = panda_datasets.PandaMeasurementDataset(
    'data/gentle_push_100.hdf5',
    subsequence_length=16,
    stddev=(0.5, 0.5),
    samples_per_pair=20,
    **dataset_args)

dynamics_recurrent_trainset = panda_datasets.PandaSubsequenceDataset(
    "data/gentle_push_100.hdf5",
    subsequence_length=32,
    **dataset_args
)


Creating dataset...
Parsed data: 1307 active, 193 inactive
Keeping (inactive): 193



Loaded 24000 points


In [5]:
# image_modality_model
image_measurement = PandaEKFMeasurementModel(missing_modalities=['gripper_pos'])
image_dynamics = PandaDynamicsModel(use_particles=False)
image_model = KalmanFilterNetwork(image_dynamics, image_measurement)

# force_modality_model
force_measurement = PandaEKFMeasurementModel(missing_modalities=['image'])
force_dynamics =  PandaDynamicsModel(use_particles= False)
force_model = KalmanFilterNetwork(force_dynamics, force_measurement)

weight_model = CrossModalWeights()

fusion_model = KalmanFusionModel(image_model, force_model, weight_model)


models ={'image': image_model, 'force': force_model, 'weight': weight_model}

In [6]:
#todo: need a different version of buddy... also probably need to load and save myself 
buddy = fannypack.utils.Buddy(experiment_name, 
                              fusion_model, 
                              optimizer_names=["im_meas", "im_dynamics", "force_dynamics", "force_ekf", "im_ekf", 
                                               "force_meas", "fusion"], 
                              load_checkpoint=True,
)

[buddy-fusion_7_n100] Using device: cuda
[buddy-fusion_7_n100] No checkpoint found


In [7]:
dataloader_dynamics = torch.utils.data.DataLoader(
    dynamics_recurrent_trainset, batch_size=32, shuffle=True, num_workers=2, drop_last=True)


for i in range(10):
    print("Training epoch", i)
    training.train_dynamics_recurrent(buddy, image_model, dataloader_dynamics, optim_name="im_dynamics")    
    print()

buddy.save_checkpoint("phase_0_im_dynamics_pretrain")

for i in range(10):
    print("Training epoch", i)
    training.train_dynamics_recurrent(buddy, force_model, dataloader_dynamics, optim_name="force_dynamics")    
    print()

buddy.save_checkpoint("phase_0_force_dynamics_pretrain")



Training epoch 0




Training epoch 1




Training epoch 2




Training epoch 3




Training epoch 4


...



Training epoch 6




Training epoch 7




Training epoch 8




Training epoch 9


[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000000400.ckpt


[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-phase_0_force_dynamics_pretrain.ckpt


In [8]:
measurement_trainset_loader = torch.utils.data.DataLoader(
    dataset_measurement,
    batch_size=32,
    shuffle=True,
    num_workers=16)

for i in range(10):
    print("Training epoch", i)
    training.train_measurement(buddy, image_model, measurement_trainset_loader, log_interval=20, optim_name="im_meas")
    print()

for i in range(10):
    print("Training epoch", i)
    training.train_measurement(buddy, force_model, measurement_trainset_loader, log_interval=20, optim_name="force_meas")
    print()

buddy.save_checkpoint("phase_2_measurement_pretrain")


Training epoch 0


[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000000500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000001000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000001500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000002000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000002500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000003000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000003500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000004000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000004500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000005000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000015500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000016000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000016500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000017000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000017500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000018000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000018500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000019000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000019500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000020000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000030500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000031000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000031500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000032000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000032500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000033000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000033500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000034000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000034500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000035000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000045500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000046000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000046500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000047000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000047500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000048000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000048500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000049000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000049500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000050000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

...

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000225500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000226000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000226500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000227000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000227500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000228000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000228500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000229000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000229500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000230000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000240500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000241000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000241500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000242000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000242500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000243000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000243500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000244000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000244500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000245000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000255500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000256000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000256500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000257000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000257500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000258000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000258500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000259000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000259500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000260000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000270500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000271000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000271500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000272000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000272500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000273000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000273500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000274000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000274500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000275000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000285500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000286000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000286500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000287000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000287500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000288000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000288500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000289000.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000289500.ckpt
[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000290000.ckpt
[buddy-fusion_7_n100] Saved checkpoint t

SyntaxError: invalid syntax (<ipython-input-9-eff1bd15fd54>, line 6)

In [12]:
e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=32, shuffle=True, num_workers=2)

for i in range(20):
    print("Training epoch", i)
    training.train_e2e(buddy, force_model, e2e_trainset_loader, optim_name="force_ekf")
    
for i in range(20):
    print("Training epoch", i)
    training.train_e2e(buddy, image_model, e2e_trainset_loader, optim_name="im_ekf")
    



Training epoch 0



Training epoch 1


[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000300500.ckpt

Training epoch 2



Training epoch 3



Training epoch 4


...


Training epoch 12



Training epoch 13


[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000302000.ckpt

Training epoch 14



Training epoch 15


KeyboardInterrupt: 

In [13]:
buddy.save_checkpoint("phase_3_e2e")

[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-phase_3_e2e.ckpt


In [14]:
for i in range(1000):
    print("Training epoch", i)
    training.train_fusion(buddy, fusion_model, e2e_trainset_loader, optim_name="fusion")
    
    
buddy.save_checkpoint("phase_4_fusion")

Training epoch 0



Training epoch 1



Training epoch 2



Training epoch 3



Training epoch 4


...


Training epoch 19


[buddy-fusion_7_n100] Saved checkpoint to path: checkpoints/fusion_7_n100-0000000000303000.ckpt

Training epoch 20



Training epoch 21



Training epoch 22


OSError: [Errno 12] Cannot allocate memory